In [1]:
from utils_others import *

In [2]:
genai_api_key=""
# 放入gemini api (我的api余额已经快用光光了.....捂脸)

In [3]:
# 模型设定
gemini_hypers = {
    # temp=[0.2, 0.4, 0.6, 0.8, 1.0],
    'temp': 0.2,  # temp 在0.2 or 0.4是比较好的，不过这个寻优只是通过MAPE,R^2找到，因为gemini没办法得到输出token的log-p，无法调用validation-likelihood function
    # 话又说回来, 似乎validation-likelihood function也不一定使得模型表现更好......
    'alpha': 0.95,
    'beta': 0.3,
    'basic': [False],
    'settings': [SerializerSettings(base=10, prec=3, signed=True,half_bin_correction=True)],
    # prec可能=3
}

model_hypers = {
    'gemini-1.0-pro': {'model': 'gemini-1.0-pro', **gemini_hypers},
    # 'gemini-pro': {'model': 'gemini-pro', **gemini_hypers}
}

model_predict_fns = {
    'gemini-1.0-pro': get_llmtime_predictions_data,  # 此处假定用gpt系列的encoding方式来进行预测
    # 'gemini-pro': get_llmtime_predictions_data
}


In [4]:
datasets = get_datasets()  
datasets_tmp = get_memorization_datasets()
datasets.update(datasets_tmp)

D:\Desktop\Works_left_over\0-已完成 (形式上)\1-Time-LLM\2-llmtime-main\llmtime-main\data1\small_context.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  series = pd.read_csv(f, index_col=0, parse_dates=True).values.reshape(-1)


In [5]:
# 一些参数设置
dataset_name = 'WineDataset'
train, test = datasets[dataset_name]

num_samples = 10  # 跑多少次


In [6]:
# out_gemini_pro = {}  # gemini-pro
# out_gemini_pro_number = {}  # gemini-1.0-pro

out_gemini_pro, out_gemini_pro_number = prediction_gemini(model_predict_fns, train, test, model_hypers, num_samples=num_samples, whether_blanket=False, dataset_name='WineDataset', genai_key=genai_api_key)

Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gemini-1.0-pro', 'temp': 0.2, 'alpha': 0.95, 'beta': 0.3, 'basic': False, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'WineDataset'}) 
 with NLL inf


  0%|          | 0/1 [00:00<?, ?it/s]

Index: 0
Index: 1
Index: 2
Index: 3


KeyboardInterrupt: 

In [ ]:
# 这里采用gemini-1.0-pro
# 绘制包含训练和测试数据的图像

plot_preds_w_train_test(train, test, out_gemini_pro_number['WineDataset'], model_name='gemini-1.0-pro', ds_name='WineDataset', show_samples=False)


In [ ]:
# 绘制包含测试数据的图像

plot_preds_w_test(test, out_gemini_pro_number['WineDataset'], model_name='gemini-1.0-pro', ds_name='WineDataset', show_samples=False)

In [ ]:
# 计算指标 (多次结果的均值)

mse_mean, mae_mean, mape_mean, r2_mean = metrics_used(test=test, dataset_name=dataset_name, original_pred=out_gemini_pro_number, num_samples=num_samples)

In [ ]:
# 超参数寻优
temp_list = [0.2, 0.4, 0.6, 0.8, 1.0]
prec_list = [2, 3]

opt_hyper_gemini(model_predict_fns, train, test, model_hypers, num_samples=num_samples, whether_blanket=False,
                      dataset_name=dataset_name, genai_key=genai_api_key, temp_list=temp_list, prec_list=prec_list)